In [1]:
import warnings
warnings.filterwarnings('ignore')

## Diagnosing the model

The Diagnose stage provides four different diagnostics. Since some of them are time-consuming, a fast diagnosis and a slower diagnosis can be run separately.  
The fast diagnosis checks if the MNLE has learned to emulate the behaviour of the simulator. Additionally, the posterior distribution inferred on test data is verified to differ from the prior distribution.
If the results of the fast diagnosis look satisfactory, the slow diagnosis determines if the posterior distribution $p(\theta | x, \pi)$ is well calibrated. See section Improve results below for information on how to improve the model performance.  
Plots are saved in the *diagnose* subfolder within the experiment`s results folder.

#### Fast diagnosis

The fast diagnosis utilizes the test data containing multiple independent and identically distributed observations per combination of inputs (i.e. parameters and experimental conditions). The number of plots that will be created per diagnosis test equals `sim_iid_test_data_params.num_params`.  

**Compare observations**

This test compares observations simulated by the DDM simulator to observations sampled from the MNLE using the same input parameters and experimental conditions. A well trained MNLE should return observations similar to the simulated observations. The following figure shows one example plot. The caption of the plot describes the input used for simulating and sampling the observations. The pink histogram shows simulated observations while the blue histogram corresponds to observations sampled from the MNLE.  
You will find the compare observation plots at *diagnose/compare_observations.png* in your result folder.

<img src="tutorial_images/compare_observations.png" width=1000px>


**[Posterior Predictive Check](https://rss.onlinelibrary.wiley.com/doi/full/10.1111/rssa.12378)**

The posterior predictive check provides a first assessment of the posterior.  
Take a look at the example plot below. On the left you can see a blue histogram describing the posterior distribution $p(\theta | x, \pi)$ of each parameter given the simulated test observations plotted on the right. The pink lines describe the ground truth parameters used for simulating these test observations. The ground truth should lie somewhere within the blue histogram, as shown below.  
The grey outline on the left visualizes an approximation of the prior. The posterior shape of a parameter should usually differ from its prior shape, especially if the prior is uniform. Otherwise, the parameter might be redundant for generating observations.    
The histogram of simulated samples on the right shows observations that have been simulated using the posterior samples. If reasonable posterior samples have been inferred from the test observations, they should allow to generate observations that are close to the test observations. Therefore, both pink histograms should look similar.

For more information about the posterior predictive check read [sbi tutorial 12](https://github.com/mackelab/sbi/blob/main/tutorials/12_diagnostics_posterior_predictive_check.ipynb).

<img src="tutorial_images/posterior_predictive.png" width=1000px>

#### Slow diagnosis

As mentioned before, the slow diagnosis checks if the posterior is well calibrated. The top left plot in the subsequent image shows you an example ground truth posterior in green. Posterior approximations based on the trained MNLE might be shifted to the left or right, this is called positive bias or negative bias. Moreover, their standard deviation might be too large (overdispersion) or too narrow (underdispersion).

**[Simulation-Based Calibration (SBC)](https://arxiv.org/abs/1804.06788)**

SBC checks if a bias, over- or underdispersion has been found in the posterior approximation. 

<div style="display:flex"><div style="border-color:rgba(102,178,255,0.75); border-style:solid; padding: 7px; border-width:2px; margin-right:9px"> 

SBC makes use of test observations $\tilde{x}_i \sim p(x| \tilde{\theta}_i)$ that have been simulated with parameters $\tilde{\theta}_i \sim p(\theta)$ drawn from the prior distribution. A set of posterior samples $\{\theta_1, ..., \theta_L\}_i \sim p(\theta | \tilde{x}_i)$ can be inferred for each test observation $\tilde{x}_i$. The posterior can be assumed to be well calibrated, if the prior sample $\tilde{\theta}_i$ that generated the test observations $\tilde{x}_i$ is uniformly distributed within the coverage of the posterior samples $\{\theta_1, ..., \theta_L\}_i$. To verify this uniform distribution, the rank of each $\tilde{\theta}_i$ is computed, i.e. the number of posterior samples $\{\theta_1, ..., \theta_L\}_i$ that are smaller than $\tilde{\theta}_i$. The ranks are then plotted against the cumulative density function of a uniform distribution, as shown in the plot below.
</div> </div>

The bottom left image visualizes the test results. A distribution is assumed to be well defined if its line lies within the grey area that represents the 99% confidence region. You can see that this applies to the ground truth plotted in green. The remaining lines serve as a reference for the discernible pattern that each of the poorly defined posteriors will produce.  

The plot on the right shows an example of the image the pipeline will create and save in *diagnose/sbc.png* in your result folder. You can observe one line for each parameter. 

Read [sbi tutorial 13](https://github.com/mackelab/sbi/blob/main/tutorials/13_diagnostics_simulation_based_calibration.ipynb) for more information about SBC.

<img src="tutorial_images/sbc.PNG" width=1100px>


**[Local coverage test](https://proceedings.mlr.press/v161/zhao21b.html)**

The local coverage test is similar to SBC, but tests the posterior distribution locally instead of globally. 

<div style="display:flex"><div style="border-color:rgba(102,178,255,0.75); border-style:solid; padding: 7px; border-width:2px; margin-right:9px"> 

In some cases, the ranks computed by SBC are not sufficient to verify the posterior calibration. The local calibration test represents the uniform cumulative density function as a linear space $[\alpha_0 = 0.05, ...  , \alpha_n = 0.95]$. For each $\alpha_i$, a Logistic Regression model is fitted using the test observations $\tilde{x}$ and their corresponding SBC ranks. The fitted model predicts a cumulative density value $\hat{\alpha}_i$ for arbitrary observations $x$. If the SBC ranks are uniform, the predicted $\hat{\alpha}_i$ should be close to the true $\alpha_i$. 
As a result, a local coverage for each $\alpha_i$ is verified.
</div> </div>

An overview of the local coverage is provided by utilizing the local results to compute a global *p*-value for each parameter. The *p*-values are saved in *diagnose/global_pvalues.json* as visualized in the figure below. A *p*-value < 0.05 indicates that the posterior is not well defined con respect to this parameter.

<img src="tutorial_images/pvalues.PNG">

The plots saved in *diagnose/local_coverage_check.png* will allow you to get a closer look at the local coverage performance. The subsequent example image visualizes local coverage tests on the posterior $p(\theta | x, \pi)$ specified in the subplot title. *diagnose/local_coverage_check.png* only contains plots showing local coverage tests that indicated a bias or over-/underdispersion. In the example plot below a slight positive bias pattern for the drift parameter is identifiable. The drift bias is in accordance with the small global p-value that has been computed for the drift parameter. If all local coverage tests of a parameter indicated the posterior being well defined, no plots are drawn for this parameter. You can choose the maximum number of plots drawn per parameter by adapting `n_local_coverage_plots` in *config/tasks*.

<img src="tutorial_images/local_coverage.png" width=1000px>

## Improve results

The easiest way to improve the model performance might lie in incrementing the training data by means of increasing the number of training simulations in *config/task* and repeating the simulation step. If the simulator is slow, see **tutorial_7** to reuse the previously simulated data and add simulations to it.

The WandB visualization allows to monitor the training performance. The blue boxes in **tutorial_3** describe how to interpret the visualization and adapt the search ranges of hyperparameters towards the best configuration. Additionally, the number of training runs can be increased via `n_wandb_sweeps` in *config/algorithm*.

The posterior result can be improved by increasing thinning, the number of chains or the number of warmup steps. Changing the initial strategy from `proposal` to `sir` might improve the initial locations of chains since not only the prior probability but also the likelihood is taken into account. When using `sir`, the number of initial candidate locations can be increased via `init_strategy_num_candidates`.  
Selecting the [No-U-Turn sampler](https://www.jmlr.org/papers/volume15/hoffman14a/hoffman14a.pdf) might yield better posteriors than Hamiltonian Monte Carlo due to needing less tuning. See **tutorial_4** for a description of the posterior parameters.


## Run fast diagnosis step




In [ ]:
dir = '../results/${result_folder}'
%run ../ddm_stride/run.py hydra.run.dir={dir} run_diagnose_fast=True

## Run slow diagnosis step



In [ ]:
dir = '../results/${result_folder}'
%run ../ddm_stride/run.py hydra.run.dir={dir} run_diagnose_slow=True